fatality rate?  total infections? total deaths?


model: fatalities?  (estimate from curves.. when do get peak, how much time has elapsed?)
infection rate?

data:
+ https://github.com/govex/COVID-19
+ https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
+ https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv (used by Johns Hopkins; county-level)


In [19]:
import pandas as pd
import datetime

In [20]:
data = pd.read_csv("data/combined_data.csv")

In [21]:
data.head()

,Unnamed: 0,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,ten plus cases
0,0,2.56,580,6009,10431,4473,11938,5507,53742,9.9,...,0,0,0,0,0,0,0.0,0,0,3/28/20
1,1,3.15,7797,3901,11372,4707,3752,4899,52644,15.6,...,0,0,0,0,1,1,1.0,1,1,4/12/20
2,2,3.31,30494,19232,37518,20736,12420,17109,190707,10.5,...,2,2,3,3,3,3,3.0,3,3,3/21/20
3,3,2.37,596,7285,9199,5133,1562,4717,44505,5.1,...,1,1,1,1,1,1,1.0,1,1,3/28/20
4,4,2.59,743,6543,17734,7999,3077,8953,69649,9.5,...,3,3,3,3,3,3,3.0,3,3,4/3/20


In [22]:
data[(data["county"] == "New York County") | (data["county"] == "Alameda County")].iloc[:,[1,2,3,4,5,6,7,8,9,10, 15, 17, 95, 105, 106, 107, 108]]

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,county,FIPS,4/4/20,4/14/20,4/15/20,4/16/20,ten plus cases
337,2.84,4405,180270,192425,72367,86012,76344,882648,5.1,48595,Alameda County,6001.0,12,23.0,36,39,3/14/20
489,2.08,1113,195150,209196,93400,22918,58324,901880,12.1,74911,New York County,36061.0,2630,7905.0,8455,11477,3/6/20


In [23]:
data.columns.values

array(['Unnamed: 0', 'household_size', 'empl_agriculture',
       'empl_professional', 'empl_social', 'empl_services',
       'empl_manufacturing', 'empl_retail', 'employed', 'prc_fam_poverty',
       'avg_income', 'prc_public_transp', 'population', 'pop_65_plus',
       'health_ins', 'county', 'state', 'FIPS', 'area',
       'domestic_passengers', 'intl_passengers', 'order started',
       '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20',
       '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20',
       '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20',
       '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20',
       '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20',
       '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20',
       '3/10/20', '3/11/20', '3/12/20', '3/13/2

### transform the data

+ convert "ten plus cases" into days since 10+ cases detected
+ convert empl_... into percent empl_.../employed
+ convert pop_65_plus and health_ins into percent pop_65_plus/population, ...

In [24]:
dt = datetime.datetime.strptime(data.loc[data["FIPS"] == 36061, "ten plus cases"].values[0], "%m/%d/%y")

In [25]:
days_elapsed = list()

for x in range(data.shape[0]):
    dt = datetime.datetime.strptime(data.iloc[x]["ten plus cases"], "%m/%d/%y")
    days_elapsed.append((datetime.date.today() - dt.date()).days)

In [26]:
data["days since 10+"] = days_elapsed

In [27]:
data = data.drop(["Unnamed: 0", "ten plus cases"], axis=1)

In [28]:
data.head()

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,days since 10+
0,2.56,580,6009,10431,4473,11938,5507,53742,9.9,27742,...,0,0,0,0,0,0,0.0,0,0,19
1,3.15,7797,3901,11372,4707,3752,4899,52644,15.6,22628,...,0,0,0,0,1,1,1.0,1,1,4
2,3.31,30494,19232,37518,20736,12420,17109,190707,10.5,30674,...,2,2,3,3,3,3,3.0,3,3,26
3,2.37,596,7285,9199,5133,1562,4717,44505,5.1,37645,...,1,1,1,1,1,1,1.0,1,1,19
4,2.59,743,6543,17734,7999,3077,8953,69649,9.5,28144,...,3,3,3,3,3,3,3.0,3,3,13


In [29]:
data.describe()

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,days since 10+
count,827.000000,8.270000e+02,8.270000e+02,8.270000e+02,8.270000e+02,8.270000e+02,8.270000e+02,8.270000e+02,827.000000,827.000000,...,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.00000
mean,2.590459,-1.207199e+06,-1.188644e+06,-1.171359e+06,-1.192938e+06,-1.193828e+06,-1.191189e+06,1.639734e+05,9.031197,32026.368803,...,16.521161,18.588875,20.916566,23.319226,25.177751,26.897219,29.438936,32.459492,37.887545,20.58283
std,0.250332,3.477348e+07,3.477415e+07,3.477478e+07,3.477399e+07,3.477395e+07,3.477405e+07,2.931048e+05,4.249275,7871.869665,...,163.433174,184.347985,208.484777,228.231976,247.166301,263.504874,283.811599,304.015992,407.392848,7.97401
min,1.900000,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,1.776300e+04,1.300000,14605.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,2.410000,5.265000e+02,3.597000e+03,1.033100e+04,4.105500e+03,4.681000e+03,5.070500e+03,4.417550e+04,5.900000,26839.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,16.00000
50%,2.550000,9.470000e+02,7.097000e+03,1.752200e+04,7.252000e+03,8.201000e+03,9.038000e+03,7.549400e+04,8.400000,30614.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,21.00000
75%,2.710000,1.719500e+03,1.867950e+04,4.039800e+04,1.613850e+04,1.616800e+04,1.866300e+04,1.686300e+05,11.400000,35698.000000,...,6.000000,7.000000,7.000000,8.000000,9.000000,9.000000,10.000000,11.000000,12.500000,26.00000
max,4.110000,6.613900e+04,6.423530e+05,1.031614e+06,5.838530e+05,4.555510e+05,5.049400e+05,5.001369e+06,29.400000,74911.000000,...,4571.000000,5150.000000,5820.000000,6367.000000,6898.000000,7349.000000,7905.000000,8455.000000,11477.000000,45.00000


In [30]:
data.columns.values

array(['household_size', 'empl_agriculture', 'empl_professional',
       'empl_social', 'empl_services', 'empl_manufacturing',
       'empl_retail', 'employed', 'prc_fam_poverty', 'avg_income',
       'prc_public_transp', 'population', 'pop_65_plus', 'health_ins',
       'county', 'state', 'FIPS', 'area', 'domestic_passengers',
       'intl_passengers', 'order started', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20',
       '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20',
       '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20',
       '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20',
       '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20',
       '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20',
       '3/12/20', '3/13/20', '3/14/20',

In [31]:
data.loc[data["days since 10+"] > 40, ["county", "4/16/20", 'household_size', 'empl_agriculture', 'empl_professional',
       'empl_social', 'empl_services', 'empl_manufacturing',
       'empl_retail', 'employed', 'prc_fam_poverty', 'avg_income',
       'prc_public_transp', 'population', 'pop_65_plus', 'health_ins', 'area', 'domestic_passengers',
       'intl_passengers', 'order started', "days since 10+"]]

,county,4/16/20,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,...,avg_income,prc_public_transp,population,pop_65_plus,health_ins,area,domestic_passengers,intl_passengers,order started,days since 10+
113,Santa Clara County,69,2.95,5486,214817,201319,79069,163171,82980,1014217,...,58588,4.1,1937570,261252,1927339,1290.10,6547000.0,471600.0,03/17/20,44
217,King County,312,2.45,6137,256817,244493,121986,104072,145781,1233478,...,52804,14.6,2233163,294891,2219544,2115.57,21381000.0,2595600.0,03/23/20,45
235,Los Angeles County,457,2.99,20331,642353,1031614,583853,455551,504940,5001369,...,34115,5.7,10105518,1375959,10034790,4057.88,34589000.0,12570600.0,03/19/20,42
489,New York County,11477,2.08,1113,195150,209196,93400,22918,58324,901880,...,74911,59.9,1628701,268834,1617657,22.83,0.0,0.0,03/22/20,41
691,Westchester County,668,2.67,1532,66081,141140,45020,16139,38273,485840,...,56893,22.8,967612,165337,955155,430.50,775000.0,0.0,03/22/20,43
822,Snohomish County,86,2.68,2594,52310,83735,36416,60517,50039,424295,...,39500,5.9,814901,109883,804955,2087.27,0.0,0.0,03/23/20,42


In [34]:
data.loc[data["4/16/20"] > 1000, ["county", "4/16/20", 'household_size', 'empl_agriculture', 'empl_professional',
       'empl_social', 'empl_services', 'empl_manufacturing',
       'empl_retail', 'employed', 'prc_fam_poverty', 'avg_income',
       'prc_public_transp', 'population', 'pop_65_plus', 'health_ins', 'area', 'domestic_passengers',
       'intl_passengers', 'order started', "days since 10+"]]

,county,4/16/20,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,...,avg_income,prc_public_transp,population,pop_65_plus,health_ins,area,domestic_passengers,intl_passengers,order started,days since 10+
287,Nassau County,1109,2.99,658,95734,200423,50732,28642,62638,694792,...,50397,17.3,1358343,241330,1348465,284.72,0.0,0.0,03/22/20,38
489,New York County,11477,2.08,1113,195150,209196,93400,22918,58324,901880,...,74911,59.9,1628701,268834,1617657,22.83,0.0,0.0,03/22/20,41


In [37]:
data.loc[data["4/16/20"] > 700, ["county", "4/16/20", 'household_size', 'empl_agriculture', 'empl_professional',
       'empl_social', 'empl_services', 'empl_manufacturing',
       'empl_retail', 'employed', 'prc_fam_poverty', 'avg_income',
       'prc_public_transp', 'population', 'pop_65_plus', 'health_ins', 'area', 'domestic_passengers',
       'intl_passengers', 'order started', "days since 10+"]]

,county,4/16/20,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,...,avg_income,prc_public_transp,population,pop_65_plus,health_ins,area,domestic_passengers,intl_passengers,order started,days since 10+
287,Nassau County,1109,2.99,658,95734,200423,50732,28642,62638,694792,...,50397,17.3,1358343,241330,1348465,284.72,0.0,0.0,03/22/20,38
357,Cook County,722,2.57,5353,385024,596333,244780,244674,244417,2585305,...,37538,19.1,5180493,757941,5132750,945.33,43405000.0,7100700.0,03/21/20,37
489,New York County,11477,2.08,1113,195150,209196,93400,22918,58324,901880,...,74911,59.9,1628701,268834,1617657,22.83,0.0,0.0,03/22/20,41
574,Wayne County,884,2.52,2503,81122,166216,75238,140598,74889,753512,...,26988,2.5,1753893,270261,1741716,612.08,15551000.0,1861100.0,03/24/20,31
